<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:60px; font-weight:bold;">CineClassify</div>
</div>
<br>
<p>In een wereld waar films in overvloed zijn, is er een groeiende behoefte om ze gemakkelijk en duidelijk te kunnen classificeren. Veel kijkers hebben geen goed overzicht over alle mengende genres en de vele nuances. Dit kan het zoeken van een leuke film voor op de vrijdagavond vanuit de Netflix catalogus of een trip naar de bios onnodig gecompliceerd maken.  

Daarom heeft het nieuwe filmplatform “CineClassify” als doel om films automatisch te laten classificeren in verschillende genres en een duidelijke gids te brengen naar de kijkers. Om dit doel te bereiken hebben ze een data-science team ingehuurd om een model te maken die genres kan gaan voorspellen. Aan de hand van verschillende gegevens, bijv. cast, regisseur, reviews, etc., moet het mogelijk worden gemaakt om filmliefhebbers over de wereld te helpen om eenvoudiger films te vinden die passen bij hun smaak. 

In dit notebook werken we aan onze opdracht van CineClassify. Er zal een pipeline worden gebouwd om de data in te laden en er een dataframe van te maken. Dit dataframe kan worden gebruikt om de data duidelijk in te zien voor het datascience team.</p>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Inhoudsopgave</div>
    <a name='begin'></a>
</div>

1. [Importeren van libaries](#start)
2. [IMDb Webscraping](#ws)
3. [Database: Movie Summaries](#db)
4. [API](#api)
5. [Preprocessing en Feature Engineering](#tr)
6. [Opzetten van de Pipeline](#pipe)
7. [Aantonen dat de Pipeline werkt](#toon)

<br>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Importeren van libaries</div>
    <a name='start'></a>
</div>

In [47]:
# Importeren standaard libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importeren webscraping libaries
import requests
from bs4 import BeautifulSoup
import regex as re

# Importeren time-out libaries
from time import sleep
from random import randint

# Importeren Database libaries
import tarfile
import os
import urllib.request
from io import BytesIO
import sqlite3

# Importeren API libabries
import requests

# Importeren Preprocessing libaries
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import json

# Importeren FE libaries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">IMDb Webscraping</div>
    <a name='ws'></a>
</div>

Om een deel van de data te krijgen is het nodig om data te verkrijgen van het internet. Dit wordt gedaan door middel van een techniek genaamd webscraping. Door middel van de BeautifulSoup library voor Python is het gemakkelijk gemaakt om deze stappen te ondernemen. Door het bekijken van de HTML code van de website kunnen de nodige elementen gevonden worden en kan de data opgehaald worden van de website. Voordat we beginnen met het coderen van de soup worden er eerst een paar nodige elementen aangemaakt.

Om deze data te verkrijgen is er gebruik gemaakt van een webscraper. Deze webscraper is helaas niet meer bruikbaar doordat de source-code van de website volledig is veranderd. De code die gebruikt is om de data te verkrijgen is in de markdown cell gezet. Na de uitleg van de code, lezen we het csv bestand in dat gemaakt is na het uitvoeren van de code.

```py
# Aanmaken van lijsten om de data in te stoppen
titel = []
jaartal = []
lengte = []
imdb_scores = []
meta_scores = []
stemmen = []
us_omzet = []
beschrijving = []
certificaat = []
genre = []
regisseur = []
sterren = []

# Verkrijgen van de engelse namen van films
en_titel = {'Accept-Language': 'en-US, en;q=0.5'}

# Aanmaken van lijst voor pagina's
pagina = np.arange(1, 1001, 50)

# Aanmaken van de URL
url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&start='

# Zorgen dat de scraping voor elke 50 gaat
for p in pagina:
    # Pakken van URL
    p = requests.get(
        url + str(p) + '&ref_=adv_nxt', headers=en_titel
        )

    # Beginnen van de soup
    soup = BeautifulSoup(p.text, 'html.parser')

    # Zoeken van alle films op de pagina
    films = soup.find_all('div', class_='lister-item mode-advanced')

    # Wachtijd van 2 tot 10 seconden
    sleep(randint(2, 10))

    for item in films:
        # Titel
        titel.append(item.h3.a.text)

        # Jaartal
        jaartal.append(item.h3.find('span', class_='lister-item-year').text)

        # Regisseur
        regisseur.append(item.find('p', class_='').find('a').text)

        # Hoofd-acteurs
        acteurs = item.find('p', class_='').find_all('a')
        stars = []
        for tag in acteurs[-4:]:
            stars.append(tag.text)
        sterren.append(stars)

        # Leeftijd certificatie
        cert = (item.find('span', class_='certificate').text
                if item.p.find('span', class_='certificate') else 'NotFound')
        certificaat.append(cert)

        # Lengte
        runtime = (item.find('span', class_='runtime').text
                    if item.p.find('span', class_='runtime') else 'NotFound')
        lengte.append(runtime)

        # Genre
        gen = (item.find('span', class_='genre').text
                if item.p.find('span', class_='genre') else 'NotFound')
        genre.append(gen)

        # IMDb rating
        imdb_scores.append(float(item.strong.text))

        # meta_scores
        m_score = (item.find('span', class_='metascore').text
                    if item.find('span', class_='metascore') else 'NotFound')
        meta_scores.append(m_score)

        # Beschrijving
        desc = item.find_all('p', class_='text-muted')
        beschrijving.append(desc[1].text)

        # Stemmen en Omzet
        so = item.find_all('span', attrs={'name':'nv'})
        stemmen.append(so[0].text)
        us_omzet.append(so[1].text if len(so) > 1 else '-')

    print("-- Iteratie van loop voltooid --")

# Aanmaken van een dataframe
films = pd.DataFrame(
    {'Titel' : titel,
     'Beschrijving' : beschrijving,
     'Regisseur' : regisseur,
     'Hoofd Acteurs' : sterren,
     'Age_Rating' : certificaat,
     'Genre' : genre,
     'Jaar' : jaartal,
     'Minuten' : lengte,
     'IMDb_Score' : imdb_scores,
     'Meta_Score' : meta_scores,
     'Stemmen' : stemmen,
     'Omzet (in M)' : us_omzet}
)

# Data preprocessing van de films dataframe
# Opschonen van de omschrijving kolom
films['Beschrijving'] = films['Omschrijving'].str.strip()

# Opschonen van de acteurs kolom
films['Hoofd Acteurs'] = films['Hoofd Acteurs'].astype(str)\
                            .replace({'\'': '', '\[|\]': ''}, regex=True)

# Opschonen van de genres kolom
films['Genre'] = films['Genre'].str.strip()

# Opschonen van de Jaar kolom
films['Jaar'] = films['Jaar'].str.extract('(\d+)').astype(int)

# Opschonen van de Minuten kolom
films['Minuten'] = films['Minuten'].str.extract('(\d+)').astype(int)

# Opschonen van de Meta_score kolom
films['Meta_Score'] = films['Meta_Score'].str.extract('(\d+)')

# Omzetten naar float en NotFound veranderen naar NaN
films['Meta_Score'] = pd.to_numeric(films['Meta_Score'], errors='coerce')

# Opschonen van de Stemmen kolom
films['Stemmen'] = films['Stemmen'].str.replace(',', '').astype(int)

# Opschonen van Omzet kolom
# Weghalen van '$' en 'M'
films['Omzet (in M)'] = films['Omzet (in M)'].map(lambda x: x.lstrip('$').rstrip('M'))

# Omzetten naar float en NotFound veranderen naar NaN
films['Omzet (in M)'] = pd.to_numeric(films['Omzet (in M)'], errors='coerce')

films.to_csv('IMDb_data.csv')
```

Met de bovenstaande code is het gelukt om de data van de voorgaande versie van IMDb te extraheren en de data alvast te preprocessen. Deze data is vervolgens in een CSV bestand gezet die nu zal worden ingeladen.

Later is besloten om enkel drie kolommen te behouden in deze dataset. Daarom missen er heel wat kolommen uit de oorspronkelijke code met het bestand. De code om deze kolommen eruit te halen is te vinden in het hoofdstuk preprocessing.

In [48]:
# Inladen van IMDb Data
webscraper = pd.read_csv('IMDb_data.csv')

# Tonen data
display(webscraper.dtypes)
display(webscraper)

Film            object
Omschrijving    object
Genres          object
dtype: object

,Film,Omschrijving,Genres
0,The Shawshank Redemption,"Over the course of several years, two convicts...",Drama
1,The Godfather,"Don Vito Corleone, head of a mafia family, dec...","Crime, Drama"
2,The Dark Knight,When the menace known as the Joker wreaks havo...,"Action, Crime, Drama"
3,Schindler's List,"In German-occupied Poland during World War II,...","Biography, Drama, History"
4,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men agai...,"Action, Adventure, Drama"
...,...,...,...
995,When Marnie Was There,"Due to 12-year-old Anna's asthma, she's sent t...","Animation, Drama, Family"
996,Control,"A profile of Ian Curtis, the enigmatic singer ...","Biography, Drama, Music"
997,Philomena,A world-weary political journalist picks up th...,"Biography, Comedy, Drama"
998,Shine,"Pianist David Helfgott, driven by his father a...","Biography, Drama, Music"


Het dataframe ziet er inmiddels anders uit dan dat in de code staat beschreven. De reden hiervoor is dat de uiteindelijk onnodige kolommen zijn verwijderd en vervolgens opnieuw het csv is aangemaakt in het hoofdstuk preprocessing. Door een klein overzichtsfoutje heeft het nieuwe csv bestand dezelfde naam gekregen als de oude en is de oude dus overschreven. Dit is door middel van deze code gebeurt:

```py
# Lijst voor kolommen om te behouden
cols_to_keep = ['Titel', 'Beschrijving', 'Genre']

# Alleen behouden van de nodige kolommen
films = films[cols_to_keep]

# Opslaan van aanpassingen in CSV
films.to_csv('IMDb_data.csv', index=False)
```
Het dataframe zal in deze staat ook gebruikt worden in de pipeline, dus daarvoor is er niks veranderd.

[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Database: Movie Summaries</div>
    <a name='db'></a>
</div>

Voor het Database onderdeel van de opgave, is er gebruik gemaakt van een online downloadbare bron genaamd de Movie Summary Corpus. Deze databron bevat verschillende informatie van ongeveer 42000 films. Er staat informatie over de verschillende karakters in de films, een uitgebreide omschrijving van het plot en metadata over de film. Voor dit onderzoek zal de data over de karakters niet nodig zijn en dus zal deze niet worden ingeladen. De onderstaande stukken uitleg zijn voor de data die wij gaan gebruiken. Met dank aan de README.txt die bij de download van de data zat, is het gemakkelijker om de data en de structuur te begrijpen. Voor het inladen van elk bestand is de README gebruikt om de kolomnamen aan te maken en de data op de juiste manier in te laden.
<br>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Bestand: plot_summaries.txt</div>
    <a name='db'></a>
</div>

Door middel van pd.read_csv is het mogelijk om text bestanden in te lezen. Bij het bekijken van het tekst bestand werd duidelijk dat de scheiding van kolommen is aangegeven door een tab. Verder is de README informatie gebruikt om de kolomnamen te maken:
- Plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia.  Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary. (Gekopieerd van README.txt)
<br>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Bestand: movie.metadata.tsv</div>
</div>

Ook bij dit tsv bestand kan er gebruik worden gemaakt van de pd.read_csv functie. Bij het lezen van de README is duidelijk neergezet hoe de structuur van dit bestand eruit ziet.

Metadata for 81,741 movies, extracted from the Noverber 4, 2012 dump of Freebase.  Tab-separated; columns: (Gekopieerd van README.txt)

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

<br>
<br>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Data inladen naar database</div>
</div>

Om de data in te kunnen lezen, wordt er gebruik gemaakt van een database. Deze database zal worden aangemaakt door de corpus van het internet te downloaden, om deze vervolgens in een database in te laden. Deze taken worden gedaan met behulp van verschillende libaries, met name urllib en sqlite3. Om al deze stappen uit te voeren worden er eerst een aantal functies aangemaakt. Elke functie heeft zijn eigen doel en samen geven ze een gestreamlinede weg naar het maken van de database.

In [49]:
def downloaden_en_extraheren(url, folder):
    """
    Deze functie download een gecomprimeerd archief van de
    opgegeven URL en extraheerd de data naar een opgegeven map.

    Parameters:
    ----------
    url : str
        De URL van het te downloaden archief.

    folder : str
        Het pad naar de doelmap waarin de
        bestanden worden geëxtraheerd.

    Returns:
    ----------
    None
    """
    # Openen van de URL en lezen van de response
    response = urllib.request.urlopen(url)
    tar_data = BytesIO(response.read())

    # Downloaden en extraheren van de bestanden
    with tarfile.open(fileobj=tar_data, mode='r:gz') as tar:
        tar.extractall(path=folder)

def dataframe_naar_database(df, tabel_naam, db_loc):
    """
    Overzetten van een Pandas DataFrame naar een
    SQLite-database tabel.

    Parameters:
    ----------
    df : pandas.DataFrame
        Het DataFrame dat moet worden overgezet.

    tabel_naam : str
        De naam van de tabel in de database.

    db_loc : str
        Het pad naar de SQLite-database.

    Returns:
    ----------
    None
    """
    # Verbinden met de database
    conn = sqlite3.connect(db_loc)

    # Aanmaken van de tabel
    df.to_sql(name=tabel_naam, con=conn, if_exists='replace', index=False)

    # Pushen van verandering naar Database en stoppen connectie
    conn.commit()
    conn.close()

def query_exe(db_loc, query):
    """
    Uitvoeren van een SQL-query op de opgegeven SQLite-database
    en de resultaten omzetten naar een Pandas DataFrame.

    Parameters:
    db_loc : str
        Het pad naar de SQLite-database.

    query : str
        De SQL-query die moet worden uitgevoerd.

    Returns:
    ----------
    df : pandas.DataFrame
        Het resultaat van de query als een DataFrame.
    """
    # Verbinden met de database
    conn = sqlite3.connect(db_loc)

    # Query inlezen tot dataframe
    df = pd.read_sql_query(query, conn)

    # Sluiten van de connectie met database
    conn.close()

    return df

Met de functies aangemaakt, kunnen we de nodige parameters definieeren om de data te downloaden en in te laden in de database. 

In [50]:
# URL download
url = "http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz"

# Folder path
folder = "MovieSummaries"

# Database naam
db_loc = 'movie_database.db'

# Kolommen van de data
kolommen = {
    "movie.metadata": ['Wikipedia_ID', 'Freebase_ID', 'Titel',
                       'Release_date', 'Box_office', 'Lengte',
                       'Talen', 'Landen', 'Genre'],
    "character.metadata": ['Wikipedia_ID', 'Freebase_ID', 'Release_date',
                           'Char_Name', 'Actor_DoB', 'Actor_gender', 'Actor_h',
                           'Actor_eth', 'Actor_Name', 'Actor_age',
                           'Freebase_map_ID', 'Freebase_ch_ID', 'Freebase_ac_ID'],
    "plot_summaries": ['Wikipedia_ID', 'Beschrijving'],
    "name.clusters": ['Name', 'Freebase_ID'],
    "tvtropes.clusters": ['Trope', 'Character_Data']
}

Met gebruik van deze vier parameters is het mogelijk om de nodige functies voor het downloaden van de data en het inladen in de database uit te gaan voeren.

In [51]:
# Uitvoeren van de download en extract stap
downloaden_en_extraheren(url, folder)

# Loopen over de folder met geextraheerde data
for root, dirs, files in os.walk(folder):
    for file in files:
        # Aanmaken van bestand locatie, type en de tabel naam
        file_path = os.path.join(root, file)
        file_type = 'tsv' if file.endswith(".tsv") else 'txt'
        tabel_naam = os.path.splitext(file)[0]

        # Excluderen van README.txt
        if file != "README.txt":
            print(f"Processen van {file_type.upper()} bestand: {file_path}")

            # Aangeven welke kolommen worden gebruikt
            custom_kols = kolommen.get(tabel_naam, None)

            # Aanmaken van dataframe
            df = pd.read_csv(file_path, sep='\t', names=custom_kols)

            # Dataframe transporteren naar database
            dataframe_naar_database(df, tabel_naam, db_loc)
            print(f"Tabel '{tabel_naam}' is toegevoegd")

Processen van TSV bestand: MovieSummaries\MovieSummaries\character.metadata.tsv
Tabel 'character.metadata' is toegevoegd
Processen van TSV bestand: MovieSummaries\MovieSummaries\movie.metadata.tsv
Tabel 'movie.metadata' is toegevoegd
Processen van TXT bestand: MovieSummaries\MovieSummaries\name.clusters.txt
Tabel 'name.clusters' is toegevoegd
Processen van TXT bestand: MovieSummaries\MovieSummaries\plot_summaries.txt
Tabel 'plot_summaries' is toegevoegd
Processen van TXT bestand: MovieSummaries\MovieSummaries\tvtropes.clusters.txt
Tabel 'tvtropes.clusters' is toegevoegd


Nu alle data als tabel is toegevoegd in de database, worden er een paar queries uitgevoerd. Op deze manier is het aan te tonen dat de code heeft gewerkt. De eerste is om aan te tonen dat het werkt, de tweede is het dataframe dat uiteindelijk gebruikt zal worden. Dit tweede dataframe zal vervolgens ook worden gebruikt om te kijken welke stappen van preprocessing en feature engineering nodig zijn.

In [52]:
# Aanmaken van query
query = """
    SELECT Wikipedia_ID,
           Titel,
           Genre
        FROM 'movie.metadata'
"""

# Aanmaken van dataframe door query in te lezen
df_test = query_exe(db_loc, query)

# Tonen van dataframe
display(df_test)

,Wikipedia_ID,Titel,Genre
0,975900,Ghosts of Mars,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,Brun bitter,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,White Of The Eye,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,A Woman in Flames,"{""/m/07s9rl0"": ""Drama""}"
...,...,...,...
81736,35228177,Mermaids: The Body Found,"{""/m/07s9rl0"": ""Drama""}"
81737,34980460,Knuckle,"{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,Another Nice Mess,"{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,The Super Dimension Fortress Macross II: Lover...,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [53]:
# Aanmaken van query
query = """
    SELECT mm.Titel,
           ps.Beschrijving,
           mm.Genre
    FROM 'movie.metadata' AS mm
        
    JOIN 'plot_summaries' AS ps
        ON mm.Wikipedia_ID = ps.Wikipedia_ID
"""

# Aanmaken van dataframe door query in te lezen
database = query_exe(db_loc, query)

# Tonen van dataframe
display(database)

,Titel,Beschrijving,Genre
0,Ghosts of Mars,"Set in the second half of the 22nd century, th...","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,White Of The Eye,A series of murders of rich young women throug...,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
2,A Woman in Flames,"Eva, an upper class housewife, becomes frustra...","{""/m/07s9rl0"": ""Drama""}"
3,The Sorcerer's Apprentice,"Every hundred years, the evil Morgana returns...","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant..."
4,Little city,"Adam, a San Francisco-based artist who works a...","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ..."
...,...,...,...
42199,The Ghost Train,{{plot}} The film opens with a Great Western e...,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th..."
42200,Mermaids: The Body Found,Two former National Oceanic Atmospheric Admini...,"{""/m/07s9rl0"": ""Drama""}"
42201,Knuckle,{{No plot}} This film follows 12 years in the ...,"{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
42202,The Super Dimension Fortress Macross II: Lover...,"The story takes place in the year 2092,The Sup...","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">API</div>
    <a name='api'></a>
</div>

Voor het API onderdeel van de opgave, is er gebruik gemaakt van een gratis move api van de website Rapid api. Deze api bevat meer dan 9 miljoen verschillende titles van films, series en afleveringen. De titels worden weekelijks geupdate en de ratings en afleveringen worden dagelijks bijgewerkt. In de api staat informatie over de films en series waaronder de cast, de awards die gewonnen zijn, het jaar dat het uitkwam, de rating (van de IMDB-website), het genre en een korte omschrijving van de film/serie. Wij gebruiken alleen de titel, het genre en een korte omschrijving van de film dus hier beneden zie je de code over hoe wij deze uit de api data hebben gehaald.

In [54]:
# Invoeren van API_URL
url = "https://moviesdatabase.p.rapidapi.com/titles"

# Invoeren van query parameters
querystring = {"limit":"50", "info":"base_info"}

# Invoeren van headers voor de RapidAPI
headers = {
	"X-RapidAPI-Key": "862efd2e3dmsh364685e1c50acb8p153999jsnd3c8b4543ef9",
	"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

In [55]:
# Ophalen van response API
response = requests.get(url, headers=headers, params=querystring)

# Aanmaken lijst voor API-Data
resultaten = []

# Itereer over elk resultaat in de 'results' lijst van de JSON-respons
for resultaat in response.json()['results']:
    # Haal de benodigde velden op
    titel = resultaat['titleText']['text'] if 'titleText' in resultaat else np.nan
    genres = [genre['text'] for genre in resultaat['genres']['genres']] if 'genres' in resultaat else np.nan
    plot = resultaat['plot']['plotText']['plainText'] if 'plot' in resultaat and resultaat['plot'] and 'plotText' in resultaat['plot'] else np.nan

    # Voeg de resultaten toe aan de lijst
    resultaten.append({
        'Titel': titel,
        'Genre': genres,
        'Beschrijving': plot
    })

# Aanmaken dataframe op basis van opgehaalde data
df_yes = pd.DataFrame(resultaten)

# verwijder de missende waardes
df_yes.dropna(subset=['Titel', 'Genre', 'Beschrijving'], inplace=True)

# Toon de DataFrame
display(df_yes)


,Titel,Genre,Beschrijving
1,Les blanchisseuses,[Short],This lost film presumably features women washi...
2,Dessinateur: Von Bismark,[Short],This lost film featured a talented sketch arti...
3,"Boxing Match; or, Glove Contest","[Short, Sport]",Stage boxing match between Sergeant-Instructor...
4,Plus fort que le maître,[Short],"Little is known about this lost film, the thir..."
7,Cripple Creek Bar-Room Scene,"[Western, Short]",A vignette of a barroom/liquor-store in the We...
11,Séance de prestidigitation,[Short],Boognish the goddemon Wants your soul to perfo...
15,L'hallucination de l'alchimiste,"[Short, Fantasy, Horror]",Misidentified as Alchimiste Parafaragaramus ou...
16,Une partie de cartes,"[Short, Biography]",In what is considered to be the first remake i...
21,Escamotage d'une dame au théâtre Robert Houdin,"[Short, Horror]",As an elegant maestro of mirage and delusion d...
22,Campement de bohémiens,"[Documentary, Short]",Very little is known of this lost film; accord...


In [56]:
url = "https://moviesdatabase.p.rapidapi.com/titles"

headers = {
    "X-RapidAPI-Key": "862efd2e3dmsh364685e1c50acb8p153999jsnd3c8b4543ef9",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}


resultaten = []

# Itereer over de paginanummers
for page_number in range(1, 21):
    
    querystring = {
        "page": str(page_number),
        "info": "base_info",
        "limit": "50"
    }

    response = requests.get(url, headers=headers, params=querystring)

    # Itereer over elk resultaat in de 'results' lijst van de JSON-respons
    for resultaat in response.json()['results']:
        # Haal de benodigde velden op
        titel = resultaat['titleText']['text'] if 'titleText' in resultaat else np.nan

        # Check if 'genres' is present and not None
        if 'genres' in resultaat and resultaat['genres']:
            # Use ', '.join() to convert the list of genres into a comma-separated string
            genres = ', '.join([genre['text'] for genre in resultaat['genres']['genres']])
        else:
            genres = np.nan

        plot = resultaat['plot']['plotText']['plainText'] if 'plot' in resultaat and resultaat['plot'] and 'plotText' in resultaat['plot'] else np.nan

        # Voeg de resultaten toe aan de lijst
        resultaten.append({
            'Titel': titel,
            'Genre': genres,
            'Beschrijving': plot
        })

api = pd.DataFrame(resultaten)

# Toon de DataFrame
display(api)

,Titel,Genre,Beschrijving
0,Les haleurs de bateaux,"Documentary, Short",NaN
1,Les blanchisseuses,Short,This lost film presumably features women washi...
2,Dessinateur: Von Bismark,Short,This lost film featured a talented sketch arti...
3,"Boxing Match; or, Glove Contest","Short, Sport",Stage boxing match between Sergeant-Instructor...
4,Plus fort que le maître,Short,"Little is known about this lost film, the thir..."
...,...,...,...
995,Il mercante di Venezia,"Short, Drama","With a friend desperate for money, a merchant ..."
996,Julius Caesar,"Short, Crime, History",Roman Senators conspire to assassinate their r...
997,The Ranchman's Feud,"Short, Western","Hiram Matthews, a western ranchman, owns an ap..."
998,The Thread of Destiny,"Romance, Short","Little Myrtle, the orphan girl of San Gabriel,..."


In [57]:
import requests
import pandas as pd
import numpy as np

url = "https://moviesdatabase.p.rapidapi.com/titles"

headers = {
    "X-RapidAPI-Key": "862efd2e3dmsh364685e1c50acb8p153999jsnd3c8b4543ef9",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

resultaten = []

# Itereer over de paginanummers
for page_number in range(1, 21):
    
    querystring = {
        "page": str(page_number),
        "info": "base_info",
        "limit": "50"
    }

    response = requests.get(url, headers=headers, params=querystring)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Itereer over elk resultaat in de 'results' lijst van de JSON-respons
        json_response = response.json()
        if 'results' in json_response:
            for resultaat in json_response['results']:
                # Haal de benodigde velden op
                titel = resultaat['titleText']['text'] if 'titleText' in resultaat else np.nan

                # Check if 'genres' is present and not None
                if 'genres' in resultaat and resultaat['genres']:
                    # Use ', '.join() to convert the list of genres into a comma-separated string
                    genres = ', '.join([genre['text'] for genre in resultaat['genres']['genres']])
                else:
                    genres = np.nan

                plot = resultaat['plot']['plotText']['plainText'] if 'plot' in resultaat and resultaat['plot'] and 'plotText' in resultaat['plot'] else np.nan

                # Voeg de resultaten toe aan de lijst
                resultaten.append({
                    'Titel': titel,
                    'Genre': genres,
                    'Beschrijving': plot
                })
        else:
            print(f"Error: 'results' not found in JSON response for page {page_number}")

    else:
        print(f"Error: Request failed with status code {response.status_code}")

# Creëer DataFrame
api = pd.DataFrame(resultaten)

# Toon de DataFrame
display(api)


,Titel,Genre,Beschrijving
0,Les haleurs de bateaux,"Documentary, Short",NaN
1,Les blanchisseuses,Short,This lost film presumably features women washi...
2,Dessinateur: Von Bismark,Short,This lost film featured a talented sketch arti...
3,"Boxing Match; or, Glove Contest","Short, Sport",Stage boxing match between Sergeant-Instructor...
4,Plus fort que le maître,Short,"Little is known about this lost film, the thir..."
...,...,...,...
995,Il mercante di Venezia,"Short, Drama","With a friend desperate for money, a merchant ..."
996,Julius Caesar,"Short, Crime, History",Roman Senators conspire to assassinate their r...
997,The Ranchman's Feud,"Short, Western","Hiram Matthews, a western ranchman, owns an ap..."
998,The Thread of Destiny,"Romance, Short","Little Myrtle, the orphan girl of San Gabriel,..."


[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Preprocessing en Feature Engineering</div>
    <a name='tr'></a>
</div>

Om te kunnen werken met de ingeladen data is het noodzakelijk dat er toepassingen op worden uitgevoerd. De eerste vorm van toepassingen heet preprocessing. Deze stap is verantwoordelijk voor het volledig opschonen van de data. De tweede vorm heet Feature engineering. Deze vorm van toepassingen slaan op het bruikbaar maken van de data voor Machine Learning. Hierbij moeten er vaak nieuwe kolommen worden aangemaakt, of moeten oude kolommen worden herschreven. In deze sectie van het notebook gaan we kijken naar welke transformaties de data nodig zal hebben om bruikbaar te zijn voor Machine Learning.

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Preprocessing</div>
</div>

Omdat veel data vanuit ruwe bronnen niet correct of compleet is, is het nodig om de data waar nodig aan te passen. Deze eerste aanpassingen zijn de stappen voor preprocessing. In dit process is het gebruikelijk dat de data volledig wordt opgeschoond en dat er eventuele technieken worden gebruikt om bepaalde data typen beter bruikbaar te maken. Doorgaande dit gedeelte van het notebook worden verschillende technieken van preprocessing gebruikt om de tekst op de juiste manier voor te bereiden op de volgende stappen. Om hiermee te beginnen zal er worden gekeken naar missende waarden in onze datasets.

In [58]:
def data_info(df):
    """
    Deze functie maakt een dataframe waarbij er een soort omschrijving komt
    over de data in elk dataframe.

    Parameters:
    ----------
    df : pd.DataFrame
        Het dataframe waarbij de kolommen worden bekeken

    Returns:
    ----------
    info : pd.DataFrame
        Een dataframe met de volgende informatie:
            - Aantal missende waarden
            - Percentage missende waarden
            - Aantal nulwaarden voor numerieke kolommen
            - Type data (numeriek of categorie)
            - Hoeveelheid categorieen
    """
    # Maak een dataframe met missende waarden
    info = pd.DataFrame(df.isnull().sum(), columns=['Missende_waarden'])

    # Voeg een kolom toe met percentage missende waarden
    info['Perc_missend'] = round(info['Missende_waarden'] / len(df) * 100, 2)

    # Voeg een kolom toe voor data type
    types = []
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):
            types.append('Numeriek')
        else:
            types.append('Categorie')
    info['Type'] = types

    # Voeg een kolom toe met aantal nulwaarden
    nulwaarden = []

    # For-Loop om te kijken voor data type
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):
            nulwaarden.append((df[col] == 0).sum())
        else:
            nulwaarden.append('-')
    info['Nulwaarden'] = nulwaarden

    # Voeg een kolom toe voor aantal categorieen
    cat_aantal = []

    # For-Loop om te kijken voor data type
    for col in df.columns:
        if info.loc[col, 'Type'] == 'Categorie':
            cat_aantal.append(df[col].nunique())
        else:
            cat_aantal.append('-')
    info['Aantal_Categorie'] = cat_aantal

    return display(info)


Aan de hand van deze functie kan er overzichtelijk worden gekeken naar de missende waarden van verschillende dataframes. Dit overzicht kan worden gebruikt om te bepalen wat we met de missende willen gaan doen.

In [59]:
# Tonen van missende waarden per dataframe
data_info(database)
data_info(webscraper)
data_info(api)

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.0,Categorie,-,39914
Beschrijving,0,0.0,Categorie,-,42196
Genre,0,0.0,Categorie,-,17851


,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Film,0,0.0,Categorie,-,994
Omschrijving,0,0.0,Categorie,-,1000
Genres,0,0.0,Categorie,-,194


,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.0,Categorie,-,985
Genre,24,2.4,Categorie,-,113
Beschrijving,378,37.8,Categorie,-,610


Er zijn gelukkig geen missende waarden aanwezig in de eerste twee dataframes. In het derde dataframe is dit wel zo. Om uiteindelijk tot nuttige voorspellingen te komen is het nodig om de beschrijving te gebruiken, om deze reden zullen de rijen met missende beschrijvingen worden verwijderd uit het dataframe. Hopelijk wordt hiermee ook het probleem van de missende genres opgelost.

In [60]:
# Verwijderen van missende waarden in 'Beschrijving'
api = api.dropna(subset=['Beschrijving'])

# Tonen van status missende waarden
data_info(api)

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.00,Categorie,-,615
Genre,4,0.64,Categorie,-,104
Beschrijving,0,0.00,Categorie,-,610


Het eerder genoemde probleem van de genres is voor een groot deel verholpen, maar nog niet helemaal. Daarom zullen deze rijen ook worden gedropped, aangezien er geen missende waarden kunnen zitten in onze target kolom.

In [61]:
# Verwijderen van missende waarden in 'Genre'
api = api.dropna(subset=['Genre'])

# Tonen van status missende waarden
data_info(api)

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.0,Categorie,-,611
Genre,0,0.0,Categorie,-,104
Beschrijving,0,0.0,Categorie,-,608


Nu deze aanpassingen zijn gemaakt, kunnen we naar de volgende stap gaan. Uit de string van genres moet alleen de eerste genoemde genre overblijven. Om dit te doen maken we gebruik van de split method. Deze method split de string in items gebaseerd op een parameter, in ons geval zal dit een komma zijn.

In [62]:
def eerste_genre(tekst, splitter):
    """
    Deze functie maakt van een string aan genres een
    enkel genre. Dit genre is degene die als eerste
    in de string voorkomt.

    Parameters:
    ----------
    tekst : str
        Een string die de genres bevat, deze bevat voor elke
        split hetzelfde te herkennen gedeelte. Bijv.: ', '.
    
    splitter : str
        Het string gedeelte waarop de tekst wordt gesplitst.
    """
    
    # Split de tekst in delen aan de hang van ', '
    genres = tekst.split(splitter)

    # Selecteer het eerste genre
    eerste_genre = genres[0]

    return eerste_genre

Het eerder genoemde probleem van de genres is voor een groot deel verholpen, maar nog niet helemaal. Daarom zullen deze rijen ook worden gedropped, aangezien er geen missende waarden kunnen zitten in onze target kolom.

In [63]:
# Verwijderen van missende waarden in 'Genre'
api = api.dropna(subset=['Genre'])

# Tonen van status missende waarden
data_info(api)

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.0,Categorie,-,611
Genre,0,0.0,Categorie,-,104
Beschrijving,0,0.0,Categorie,-,608


Nu deze aanpassingen zijn gemaakt, kunnen we naar de volgende stap gaan. Uit de string van genres moet alleen de eerste genoemde genre overblijven. Om dit te doen maken we gebruik van de split method. Deze method split de string in items gebaseerd op een parameter, in ons geval zal dit een komma zijn.

In [16]:
def eerste_genre(tekst, splitter):
    """
    Deze functie maakt van een string aan genres een
    enkel genre. Dit genre is degene die als eerste
    in de string voorkomt.

    Parameters:
    ----------
    tekst : str
        Een string die de genres bevat, deze bevat voor elke
        split hetzelfde te herkennen gedeelte. Bijv.: ', '.
    
    splitter : str
        Het string gedeelte waarop de tekst wordt gesplitst.
    """
    
    # Split de tekst in delen aan de hang van ', '
    genres = tekst.split(splitter)

    # Selecteer het eerste genre
    eerste_genre = genres[0]

    return eerste_genre

Aan de hand van deze functie zal het mogelijk zijn om alle strings in het dataframe in een keer te splitsen. Doordat we apply kunnen gebruiken in combinatie met lambda is deze handeling snel en effectief uit te voeren.

In [17]:
# Toepassen van eerste genre functie
webscraper['Genre'] = webscraper['Genre'].apply(lambda x: eerste_genre(x, ', '))

# Tonen van de verandering
display(webscraper.head())

KeyError: 'Genre'

Nu het eerste genre is bepaald voor de scraper data, kan er worden gewerkt aan de database data. Deze zit iets ingewikkelder in elkaar, om hieraan te werken is het belangrijk om eerst te kijken wat er precies aan de hand is.

In [64]:
# Tonen van de eerste vijf regels van df
display(database.head())

# Bekijken van type data in elke kolom
for kolom in database.columns:
    # Ophalen van type data
    typ = type(database[kolom][0])
    
    # Tonen van kolom naam en type data
    print(f'De kolom "{kolom}" heeft data-type {typ}')

,Titel,Beschrijving,Genre
0,Ghosts of Mars,"Set in the second half of the 22nd century, th...","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,White Of The Eye,A series of murders of rich young women throug...,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
2,A Woman in Flames,"Eva, an upper class housewife, becomes frustra...","{""/m/07s9rl0"": ""Drama""}"
3,The Sorcerer's Apprentice,"Every hundred years, the evil Morgana returns...","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant..."
4,Little city,"Adam, a San Francisco-based artist who works a...","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ..."


De kolom "Titel" heeft data-type <class 'str'>
De kolom "Beschrijving" heeft data-type <class 'str'>
De kolom "Genre" heeft data-type <class 'str'>


De kolommen die kunnen worden opgeschoond zijn de 'Film' kolom en de 'Genres' kolom. De 'Film' kolom zal later worden opgeschoond met een groter aantal tekst gebaseerde kolommen die zich in elke dataset bevind. De 'Genres' kolom zal worden opgeschoond door de values uit de dictionary te halen. Om dit te doen zullen we eerst de dictionaries goed moeten neerzetten, aangezien deze nu als string in het dataframe staan.

In [65]:
# Omzetten van de strings naar dictionaries
database['dicts'] = database['Genre'].apply(json.loads)

# Tonen van het dataframe
display(database.head())

,Titel,Beschrijving,Genre,dicts
0,Ghosts of Mars,"Set in the second half of the 22nd century, th...","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","{'/m/01jfsb': 'Thriller', '/m/06n90': 'Science..."
1,White Of The Eye,A series of murders of rich young women throug...,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...","{'/m/01jfsb': 'Thriller', '/m/0glj9q': 'Erotic..."
2,A Woman in Flames,"Eva, an upper class housewife, becomes frustra...","{""/m/07s9rl0"": ""Drama""}",{'/m/07s9rl0': 'Drama'}
3,The Sorcerer's Apprentice,"Every hundred years, the evil Morgana returns...","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","{'/m/0hqxf': 'Family Film', '/m/01hmnh': 'Fant..."
4,Little city,"Adam, a San Francisco-based artist who works a...","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","{'/m/06cvj': 'Romantic comedy', '/m/0hj3n0w': ..."


Nu dit is gedaan kan de dictionary worden ingelezen met een lambda functie. Deze functie zal ervoor zorgen dat de genres als een string worden ingeladen. Door dit te doen is het mogelijk om de eerste_genre functie toe te passen om ook hier alleen het eerste genre te laten staan.

In [66]:
# Ophalen van de genres uit de dictionary
database['Genre'] = database['dicts'].apply(lambda x: ', '.join(x.values()) if isinstance(x, dict) else x)

# Toepassen van eerste_genre
database['Genre'] = database['Genre'].apply(lambda x: eerste_genre(x, ', '))

# Droppen van originele kolommen
database = database.drop(columns=['dicts'])

# Tonen van nieuw dataframe en van de informatie
display(database.head())
data_info(database)

,Titel,Beschrijving,Genre
0,Ghosts of Mars,"Set in the second half of the 22nd century, th...",Thriller
1,White Of The Eye,A series of murders of rich young women throug...,Thriller
2,A Woman in Flames,"Eva, an upper class housewife, becomes frustra...",Drama
3,The Sorcerer's Apprentice,"Every hundred years, the evil Morgana returns...",Family Film
4,Little city,"Adam, a San Francisco-based artist who works a...",Romantic comedy


,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.0,Categorie,-,39914
Beschrijving,0,0.0,Categorie,-,42196
Genre,0,0.0,Categorie,-,265


Als laatste is de API data aan de beurt. In deze data was geen anomaly te vinden na het zien van de eerste vijf regels na het inladen. Daarom zal deze niet apart worden bekeken.

In [67]:
# Toepassen van eerste genre functie
api['Genre'] = api['Genre'].apply(lambda x: eerste_genre(x, ', '))

# Tonen van de verandering
display(api.head())

,Titel,Genre,Beschrijving
1,Les blanchisseuses,Short,This lost film presumably features women washi...
2,Dessinateur: Von Bismark,Short,This lost film featured a talented sketch arti...
3,"Boxing Match; or, Glove Contest",Short,Stage boxing match between Sergeant-Instructor...
4,Plus fort que le maître,Short,"Little is known about this lost film, the thir..."
7,Cripple Creek Bar-Room Scene,Western,A vignette of a barroom/liquor-store in the We...


Voordat we gaan beginnen aan het verwerken van de tekst, is het ons opgevallen dat er minder items in titel staan dan in beschrijving. Er zal dus even worden gekeken naar eventuele duplicaten.

In [68]:
# Aanmaken lijst met alle dataframes
dataframes = [database, webscraper, api]

for data in dataframes:
    # Kijken of er meer dan 1 van een titel voorkomt
    duplicaten = data['Titel'].value_counts()[data['Titel'].value_counts() > 1]

    # Tonen van herhaalde titels
    print(f'Herhaalde titels:')
    display(duplicaten)

Herhaalde titels:


The Three Musketeers    9
Dracula                 8
Alice in Wonderland     8
Hero                    8
Madame X                7
                       ..
Between the Lines       2
Thanksgiving            2
Lost Horizon            2
Thunderbolt             2
Happy Birthday          2
Name: Titel, Length: 1830, dtype: int64

Herhaalde titels:


Scarface                           2
Beauty and the Beast               2
Drishyam 2                         2
The Girl with the Dragon Tattoo    2
All Quiet on the Western Front     2
Drishyam                           2
Name: Titel, dtype: int64

Herhaalde titels:


Hamlet                     3
The Great Train Robbery    2
Othello                    2
Baignade en mer            2
Macbeth                    2
Behind the Scenes          2
Name: Titel, dtype: int64

Zoals er te zien is, zijn er 9 films die the Three Musketeers heten. Het is belangrijk om er eerst naar te kijken voordat we doorgaan met bepalen van de handeling.

In [69]:
# Tonen van alle rijen met The Three Musketeers
database[database['Titel'].str.contains('Three Musketeers')]

,Titel,Beschrijving,Genre
1229,The Three Musketeers,When Lt. Wayne is framed for the murder of his...,Action
6224,D'Artagnan and Three Musketeers,"The film consists of three parts: *Part I: ""At...",Musical
9878,The Three Musketeers,Young d'Artagnan leaves his parents and travel...,Adventure
10548,The Three Musketeers,Callow youth D'Artagnan sets off from Gascony...,Action
11349,Doraemon: Nobita and Fantastic Three Musketeers,"Tired of constantly having nightmares, Nobita ...",Japanese Movies
13603,The Three Musketeers,{{plot}} The young d'Artagnan wants to be a mu...,Japanese Movies
19342,Barbie and the Three Musketeers,Corinne is a country girl from Gascony who dre...,Family Film
20486,The Three Musketeers,"In France during the mid-19th century, Cardina...",Family Film
24493,The Three Musketeers,The young d'Artagnan arrives in Paris with dre...,Swashbuckler films
29586,The Three Musketeers,"D'Artagnan , an inexperienced Gascon youth, tr...",Swashbuckler films


Zoals er te zien is zijn er verschillen in zowel het genre als de exacte beschrijvingen. Echter is het volgens ons handiger om eventuele herhaalde film titels niet uit de dataset te verwijderen. Hoewel dit ook kan leiden tot van foute voorspellingen, is het volgens ons belangrijker om de data zo te houden. Voornamelijk omdat verschillende versies van deze verhalen aantrekkelijk zijn voor andere doelgroepen. Zo is er hier bijvoorbeeld een musical, een familie film en wat actie films. Deze genres trekken vaak andere groepen aan, waardoor het belangrijk kan zijn om de data zo inclusief mogelijk te houden.

Nu alle dataframes opgeschoond zijn, kan de tekst worden geprocessed. Deze taak is van belang omdat het anders erg lastig is voor een ML-algoritme om dezelfde woorden te blijven herkennen. In ons geval passen wij de volgende technieken toe om de tekst duidelijk te maken:

**Stopwoorden verwijderen**<br>
Met deze techniek worden veel gebruikte stopwoorden in de engelse taal verwijderd uit de tekst. Dit wordt gedaan om minder ruis te hebben in de tekst door te zorgen dat het algoritme niet hoeft te focussen op worden zoals the of and.

**Punctuatie verwijderen**<br>
Deze techniek wordt gebruikt om alle vormen van interpunctie uit de tekst te halen. Hoewel dit soms van belang kan zijn, is het bij filmplots vaak ruis in de tekst.

**Lowercase**<br>
Deze techniek zorgt ervoor dat alle woorden worden omgezet in lowercase versies. Dit voorkomt, bijvoorbeeld, dat het woord Auto verschillend is van auto.

**Stemming**<br>
Deze techniek brengt alle woorden, waar mogelijk, terug naar de stam van het woord. Woorden zoals lopen worden loop en rennen wordt ren. Dit zorgt ervoor dat alle verschillende vormen van de woorden hetzelfde worden begrepen door het algoritme en geeft een duidelijkere tekst weer. Deze optie is gekozen over lemmetization omdat lemmetization ervoor kan zorgen dat de betekenis van woorden verloren kan gaan.

In [70]:
def process_text_columns(df, kolom):
    """
    Deze functie processed de opgegeven kolommen, zodat de
    tekst bruikbaarder is. De opgegeven kolommen moeten
    hiervoor wel tekstuele data bevatten.

    Parameters:
    ----------
    df : pandas.DataFrame
        Het dataframe waarop de aanpassingen worden uitgevoerd.

    kolom : list or str
        De kolom(men) waarop de aanpassingen worden uitgevoerd.

    Returns:
    ----------
    df_nlp : pandas.DataFrame
        Het dataframe waarop de toepassing zijn uitgevoert.
    """
    # Het selecteren van engelse stopwoorden voor in de tekst
    stopwoorden = set(stopwords.words('english'))

    def process_text(text):
        """
        Deze functies past verschillende taken
        toe op basis van NLP technieken

        Parameters:
        ----------
        text : str
            Een str aan tekst in een dataframe kolom

        Returns:
        ----------
        processed_text : str
            Een str met de aangepaste tekst
        """
        # Aanmaken van tokens in de tekst
        tokens = word_tokenize(text)

        # Het weghalen van punctuatie binnen de tekst
        no_punctuations = ' '.join(re.sub(r'\W', ' ', token) for token in tokens)

        # Het veranderen van alle tekst naar kleine letters
        lower_text = no_punctuations.lower()

        # Het verwijderen van de stopwoorden
        geen_stop = ' '.join(word for word in lower_text.split() if word not in stopwoorden)

        # Initieren van PorterStemmer
        porter = PorterStemmer()

        # Toepassen van stemming
        processed_text = ' '.join(porter.stem(word) for word in word_tokenize(geen_stop))

        return processed_text
    
    # Toepassen van tekst processing
    df[[kolom]] = df[[kolom]].apply(lambda col: col.apply(process_text))

    return df

Met de functie is het mogelijk om deze toepassingen snel uit te voeren op meerdere kolommen in de datasets.

In [71]:
# Invoeren van de kolom die aangepast worden
kolom = 'Beschrijving'

# Uitvoeren van de aanpassingen
df_db = process_text_columns(database, kolom)

# Tonen van het aangepaste dataframe
display(df_db)

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,1000,2.28,Categorie,-,40459
Beschrijving,1000,2.28,Categorie,-,42803
Genre,1000,2.28,Categorie,-,267
Film,42822,97.72,Categorie,-,994
Omschrijving,42822,97.72,Categorie,-,1000
Genres,42822,97.72,Categorie,-,194


Nu we kunnen zien dat de functie goed zijn werk doet, kan deze worden toegepast op alle andere dataframes. Beginnend met de films dataset.

In [72]:
# Uitvoeren van de aanpassingen
df_ws = process_text_columns(webscraper, kolom)

# Tonen van het aangepaste dataframe
display(df_ws)

,Titel,Beschrijving,Genre
0,The Shawshank Redemption,cours sever year two convict form friendship s...,Drama
1,The Godfather,vito corleon head mafia famili decid hand empi...,Crime
2,The Dark Knight,menac known joker wreak havoc chao peopl gotha...,Action
3,Schindler's List,german occupi poland world war ii industrialis...,Biography
4,The Lord of the Rings: The Return of the King,gandalf aragorn lead world men sauron armi dra...,Action
...,...,...,...
995,When Marnie Was There,due 12 year old anna asthma sent stay rel guar...,Animation
996,Control,profil ian curti enigmat singer joy divis whos...,Biography
997,Philomena,world weari polit journalist pick stori woman ...,Biography
998,Shine,pianist david helfgott driven father teacher b...,Biography


Nu deze is getransformeerd kunnen we de code laten werken aan de API dataset.

In [73]:
# Uitvoeren van de aanpassingen
df_api = process_text_columns(api, kolom)

# Tonen van het aangepaste dataframe
display(df_api)

,Titel,Genre,Beschrijving
1,Les blanchisseuses,Short,lost film presum featur women wash cloth
2,Dessinateur: Von Bismark,Short,lost film featur talent sketch artist draw pru...
3,"Boxing Match; or, Glove Contest",Short,stage box match sergeant instructor barrett se...
4,Plus fort que le maître,Short,littl known lost film third creat georg méliè ...
7,Cripple Creek Bar-Room Scene,Western,vignett barroom liquor store west plot per se ...
...,...,...,...
995,Il mercante di Venezia,Short,friend desper money merchant take loan ruthles...
996,Julius Caesar,Short,roman senat conspir assassin ruler
997,The Ranchman's Feud,Short,hiram matthew western ranchman own appl orchar...
998,The Thread of Destiny,Romance,littl myrtl orphan girl san gabriel stand wind...


Nu ze allemaal een verwerkte tekst hebben is het belangrijk om ze samen te voegen voordat we features aan gaan maken. Dit is om te voorkomen dat er vervolgens te veel features gaan ontstaan door het koppelen van 3 verschillende datasets. Dit zal gebeuren door de dataframes op elkaar te gaan stacken, dit houdt in dat ze op elkaar gestapeled worden. Dit is mogelijk omdat we bij het inladen van de dataset hebben gelet op welke kolommen we wilden gebruiken en hoe we deze noemden.

In [74]:
# Het 'stacken' van de dataframes
stacked_df = pd.concat(dataframes, ignore_index=True)

# Tonen van het resultaat
data_info(stacked_df)
display(stacked_df.head())

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.0,Categorie,-,40736
Beschrijving,0,0.0,Categorie,-,43802
Genre,0,0.0,Categorie,-,269


,Titel,Beschrijving,Genre
0,Ghosts of Mars,set second half 22nd centuri film depict mar p...,Thriller
1,White Of The Eye,seri murder rich young women throughout arizon...,Thriller
2,A Woman in Flames,eva upper class housewif becom frustrat leav a...,Drama
3,The Sorcerer's Apprentice,everi hundr year evil morgana return claim fin...,Family Film
4,Little city,adam san francisco base artist work cab driver...,Romantic comedy


Nu dit is gedaan kunnen we door met Feature Engineering.

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Feature Engineering</div>
</div>

Nu alle gegevens zijn gepreprocessed, kan er worden gewerkt aan het maken van de features. Deze stap, genaamd feature engineering, is eigenlijk een stap die de diepte ingaat ten opzichte van preprocessing. Een groot verschil tussen de twee stappen is dat bij Feature Engineering deze kolommen niet meer worden aangepast voor begrip, maar om het model beter te laten presteren. Dit houdt onder andere in dat ruwe data zoals tekst, video of audio omgezet gaan worden van bruikbaar naar nuttig. Om met de features in onze data te beginnen zal er gewerkt worden aan de TF-IDF feature.

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:15px; font-weight:bold;">TF-IDF (Term Frequency - Inverse Document Frequency)</div>
</div>

TF-IDF is een statistische maatstaf die het belang van een woord in een tekst berekent. Het is gebaseerd op twee componenten: Term Frequency (TF) en Inverse Document Frequency (IDF).

Term Frequency (TF): Dit geeft aan hoe vaak een woord voorkomt in een document. Het wordt berekend door het aantal keren dat een woord voorkomt te delen door het totale aantal woorden in het document. Dit wordt weergegeven als $TF(t, d)$, waarbij t de term of woord is en d het document of de tekst.

Inverse Document Frequency (IDF): Dit geeft het omgekeerde van de frequentie van het woord over alle documenten in de dataset aan. Het wordt berekend door het totale aantal documenten te delen door het aantal documenten waarin het woord voorkomt, en de uitkomst te logaritmeren. Dit uit zich in de formule als $IDF(t)$, dit staat eigenlijk voor $\log{\frac{1 + n}{1 + df(d, t)}}+1$. Hierbij is $n$ het aantal documenten en $df(d, t)$ het aantal documenten waar de term in voorkomt.

Dit geheel uit zich dan in de volgende formule:

$TFIDF = TF(t, d) × IDF(t)$

Met TF-IDF kunnen we het belang van woorden in een tekst begrijpen en benadrukken, wat nuttig is voor taken zoals tekstclassificatie, clustering en informatieherwinning. Door dit te gebruiken met ondersteuning van Truncated SVD, is het mogelijk om het aantal features ook te beperken. Dit is van belang omdat de code anders te veel geheugen gebruikt omdat er te veel feature kolommen worden aangemaakt.

In [75]:
def tfidf_features(df, kolom, componenten=100):
    """
    Voert TF-IDF vectorisatie uit en reduceert
    de dimensionaliteit met Truncated SVD.

    Parameters
    ----------
    df : pandas.DataFrame
        Het DataFrame met de tekstuele documenten.
    kolom : str
        De naam van de kolom met de tekstuele documenten.
    num_components : int, optional
        Het aantal componenten voor Truncated SVD. Standaard is 100.

    Returns
    -------
    pandas.DataFrame
        Het input DataFrame met toegevoegde gereduceerde TF-IDF functies.
    """
    # Haal de tekst op uit kolom
    teksten = df[kolom]

    # TF-IDF Vectorisatie
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(teksten)

    # Truncated SVD voor Dimensionaliteitsreductie
    svd = TruncatedSVD(n_components=componenten, random_state=42)
    red_tfidf = svd.fit_transform(tfidf_matrix)

    # Maak een DataFrame met TF-IDF kolommen
    tfidf_cols = [f"tfidf_{i}" for i in range(componenten)]
    df_tfidf = pd.DataFrame(red_tfidf, columns=tfidf_cols)

    # voeg data samen
    data = pd.concat([df, df_tfidf], axis=1)

    return data

# Apply the function to add reduced TF-IDF features
data = tfidf_features(stacked_df, kolom="Beschrijving", componenten=100)

# Display the DataFrame with added features
display(data.head())

,Titel,Beschrijving,Genre,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
0,Ghosts of Mars,set second half 22nd centuri film depict mar p...,Thriller,0.146294,-0.005193,-0.104860,0.041550,-0.032685,0.007809,0.001956,...,-0.008753,-0.004105,-0.020445,-0.022612,-0.016497,-0.017306,-0.059370,-0.009217,-0.001642,-0.023422
1,White Of The Eye,seri murder rich young women throughout arizon...,Thriller,0.153784,0.008975,-0.031865,-0.047427,-0.013926,-0.055703,-0.008560,...,-0.001120,-0.008300,-0.002532,0.022074,0.006993,-0.006014,-0.022724,-0.007042,0.006923,0.003366
2,A Woman in Flames,eva upper class housewif becom frustrat leav a...,Drama,0.243657,0.009673,0.030774,-0.047796,0.044533,-0.042187,0.017850,...,0.038003,0.026074,0.036594,0.017129,-0.002144,-0.011413,-0.013548,0.020719,-0.018267,0.010748
3,The Sorcerer's Apprentice,everi hundr year evil morgana return claim fin...,Family Film,0.120472,-0.012610,-0.004946,0.005927,0.004616,0.041906,-0.029869,...,-0.001542,-0.002407,0.038852,0.016566,0.023959,0.012314,-0.012945,-0.003089,-0.010184,-0.019493
4,Little city,adam san francisco base artist work cab driver...,Romantic comedy,0.161932,-0.008109,0.047034,-0.054677,0.023335,-0.037189,-0.033769,...,0.056913,-0.020431,0.049271,-0.032481,0.102570,-0.012483,0.090850,0.098281,-0.134997,0.048495


<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Opzetten van de Pipeline</div>
    <a name='pipe'></a>
</div>

Om te zorgen dat alle data op de juiste manier word ingeladen en getransformeerd maken we gebruik van een pipeline. De pipeline is eigenlijk een class vol met functies die ons helpt om data gemakkelijk in te laden en te transformeren. De functies in de pipeline zijn zo robuust mogelijk opgesteld, zodat er geen problemen zijn bij het inladen en verwerken van een andere databron.

In [117]:
class ETL_Pipeline:
    """
    Class voor een Extract-Transform-Load Pipeline.
    Deze class kan nodige data van een database, API of
    webscraping bron halen. Deze data kan vervolgens
    getransformeerd worden, waarna de data kan worden
    ingeladen in een pandas DataFrame.
    """
    def __init__(self, db_source=None, query=None, api_source=None, csv_file=None, headers=None):
        """
        Initiator van de class. De initiator neemt bepaalde
        waarden op. Met behulp van deze waarden kunnen de
        functies worden uitgevoerd.
        """
        self.db_source = db_source
        self.query = query
        self.api_source = api_source
        self.csv_file = csv_file
        self.data_frame = pd.DataFrame()
        self.headers = headers

    def tekst_preprocessing(df, kolom):
        """
        Deze functie processed de opgegeven kolommen, zodat de
        tekst bruikbaarder is. De opgegeven kolommen moeten
        hiervoor wel tekstuele data bevatten.

        Parameters:
        ----------
        df : pandas.DataFrame
            Het dataframe waarop de aanpassingen worden uitgevoerd.

        kolom : list or str
            De kolom(men) waarop de aanpassingen worden uitgevoerd.

        Returns:
        ----------
        df_nlp : pandas.DataFrame
            Het dataframe waarop de toepassing zijn uitgevoert.
        """
        # Het selecteren van engelse stopwoorden voor in de tekst
        stopwoorden = set(stopwords.words('english'))

        def process_text(text):
            """
            Deze functies past verschillende taken
            toe op basis van NLP technieken

            Parameters:
            ----------
            text : str
                Een str aan tekst in een dataframe kolom

            Returns:
            ----------
            processed_text : str
                Een str met de aangepaste tekst
            """
            # Aanmaken van tokens in de tekst
            tokens = word_tokenize(text)

            # Het weghalen van punctuatie binnen de tekst
            no_punctuations = ' '.join(re.sub(r'\W', ' ', token) for token in tokens)

            # Het veranderen van alle tekst naar kleine letters
            lower_text = no_punctuations.lower()

            # Het verwijderen van de stopwoorden
            geen_stop = ' '.join(word for word in lower_text.split() if word not in stopwoorden)

            # Initieren van PorterStemmer
            porter = PorterStemmer()

            # Toepassen van stemming
            processed_text = ' '.join(porter.stem(word) for word in word_tokenize(geen_stop))

            return processed_text
        
        # Toepassen van tekst processing
        df[[kolom]] = df[[kolom]].apply(lambda col: col.apply(process_text))

        return df

    def eerste_genre(tekst, splitter):
        """
        Deze functie maakt van een string aan genres een
        enkel genre. Dit genre is degene die als eerste
        in de string voorkomt.

        Parameters:
        ----------
        tekst : str
            Een string die de genres bevat, deze bevat voor elke
            split hetzelfde te herkennen gedeelte. Bijv.: ', '.
        
        splitter : str
            Het string gedeelte waarop de tekst wordt gesplitst.
        """
        
        # Split de tekst in delen aan de hang van ', '
        genres = tekst.split(splitter)

        # Selecteer het eerste genre
        eerste_genre = genres[0]

        return eerste_genre

    def tfidf_features(df, kolom, componenten=100):
        """
        Voert TF-IDF vectorisatie uit en reduceert
        de dimensionaliteit met Truncated SVD.

        Parameters
        ----------
        df : pandas.DataFrame
            Het DataFrame met de tekstuele documenten.
        kolom : str
            De naam van de kolom met de tekstuele documenten.
        num_components : int, optional
            Het aantal componenten voor Truncated SVD. Standaard is 100.

        Returns
        -------
        pandas.DataFrame
            Het input DataFrame met toegevoegde gereduceerde TF-IDF functies.
        """
        # Haal de tekst op uit kolom
        teksten = df[kolom]

        # TF-IDF Vectorisatie
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(teksten)

        # Truncated SVD voor Dimensionaliteitsreductie
        svd = TruncatedSVD(n_components=componenten, random_state=42)
        red_tfidf = svd.fit_transform(tfidf_matrix)

        # Maak een DataFrame met TF-IDF kolommen
        tfidf_cols = [f"tfidf_{i}" for i in range(componenten)]
        df_tfidf = pd.DataFrame(red_tfidf, columns=tfidf_cols)

        # voeg data samen
        data = pd.concat([df, df_tfidf], axis=1)

        return data

    def extract_db(self):
        """
        Deze functie haalt data op uit een lokale database.
        """
        # Verbinden met de database
        conn = sqlite3.connect(self.db_source)

        # Uitvoeren van de query op de database
        db_data = pd.read_sql_query(self.query, conn)

        # Sluiten van de verbinding met de database
        conn.close()

        return db_data

    def extract_api(self):
        """
        Deze functie haalt data op uit ruwe bronnen.
        """
        resultaten = []

        # Itereer over de paginanummers
        for page_number in range(1, 21):
            
            querystring = {
                "page": str(page_number),
                "info": "base_info",
                "limit": "50"
            }

            response = requests.get(url, headers=headers, params=querystring)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Itereer over elk resultaat in de 'results' lijst van de JSON-respons
                json_response = response.json()
                if 'results' in json_response:
                    for resultaat in json_response['results']:
                        # Haal de benodigde velden op
                        titel = resultaat['titleText']['text'] if 'titleText' in resultaat else np.nan

                        # Check if 'genres' is present and not None
                        if 'genres' in resultaat and resultaat['genres']:
                            # Use ', '.join() to convert the list of genres into a comma-separated string
                            genres = ', '.join([genre['text'] for genre in resultaat['genres']['genres']])
                        else:
                            genres = np.nan

                        plot = resultaat['plot']['plotText']['plainText'] if 'plot' in resultaat and resultaat['plot'] and 'plotText' in resultaat['plot'] else np.nan

                        # Voeg de resultaten toe aan de lijst
                        resultaten.append({
                            'Titel': titel,
                            'Beschrijving': plot,
                            'Genre': genres
                        })
                else:
                    print(f"Error: 'results' not found in JSON response for page {page_number}")

            else:
                print(f"Error: Request failed with status code {response.status_code}")

        api_data = pd.DataFrame(resultaten)

        return api_data

    def extract_csv(self):
        """
        Deze functie haalt data op uit ruwe bronnen.
        """
        # Inlezen van csv data
        csv_data = pd.read_csv(self.csv_file)

        return csv_data

    def transform_data(self, data_frames):
        def process_genre(x):
            try:
                genre_dict = json.loads(x)
                return ', '.join(genre_dict.values()) if isinstance(genre_dict, dict) else x
            except (json.JSONDecodeError, AttributeError):
                return x

        def process_dataframe(dataframe):
            dataframe.dropna(subset=['Beschrijving', 'Genre', 'Titel'], inplace=True)
            dataframe['Genre'] = dataframe['Genre'].apply(process_genre)
            dataframe['Genre'] = dataframe['Genre'].apply(lambda x: self.eerste_genre(x, ', '))
            dataframe.drop(columns=['dicts'], inplace=True, errors='ignore')
            return dataframe

        data_frames = [process_dataframe(df) for df in data_frames]

        stacked_data = pd.concat(data_frames, ignore_index=True)
        stacked_data.dropna(subset=['Beschrijving'], inplace=True)

        preprocessed_data = self.tekst_preprocessing(stacked_data, 'Beschrijving')
        transformed_data = self.tfidf_features(preprocessed_data, kolom="Beschrijving", componenten=100)

        return transformed_data

    def load_data(self):
        """
        Deze functie laad de data in naar een
        pandas.DataFrame.
        """
        # Maken van lege lijst voor databronnen
        all_data_frames = []

        # Extraheer data van gebruikte bronnen
        if self.db_source:
            db_data = self.extract_db()
            all_data_frames.append(db_data)
        if self.api_source:
            api_data = self.extract_api()
            all_data_frames.append(api_data)
        if self.csv_file:
            csv_data = self.extract_csv()
            all_data_frames.append(csv_data)

        # Transformeer en merge alle data
        transformed_data = self.transform_data(all_data_frames)

        # Toewijzen van de data aan de data_frame class attribuut
        self.data_frame = transformed_data

        return transformed_data

Nu de Pipeline is aangemaakt, kunnen de parameters worden gedefined. De parameters zijn opzettelijk los neergezet, om de Pipeline niet vast te zetten in het geval van kleine aanpassingen aan de locaties van bestanden.

In [118]:
# Invoeren van de DataBase naam
db_source = "movie_database.db"

# Invoeren van query op DataBase
query = """
    SELECT mm.Titel,
           ps.Beschrijving,
           mm.Genre
    FROM 'movie.metadata' AS mm
        
    JOIN 'plot_summaries' AS ps
        ON mm.Wikipedia_ID = ps.Wikipedia_ID
"""

# Invoeren van de API link
api_source = "https://moviesdatabase.p.rapidapi.com/titles"

querystring = {"limit":"50","info":"base_info"}

headers = {
    "X-RapidAPI-Key": "862efd2e3dmsh364685e1c50acb8p153999jsnd3c8b4543ef9",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

# Invoeren van csv file path
csv_file = "IMDb_data.csv"

Met alle nodige parameters gedefineerd, kan de pipeline worden gebruikt om de data in te laden.

In [119]:
etl = ETL_Pipeline(db_source=db_source,
                   query=query,
                   api_source=api_source,
                   csv_file=csv_file,
                   headers=headers)

df = etl.load_data()

In [120]:
data_info(df)

,Missende_waarden,Perc_missend,Type,Nulwaarden,Aantal_Categorie
Titel,0,0.0,Categorie,-,40736
Beschrijving,0,0.0,Categorie,-,43802
Genre,0,0.0,Categorie,-,269
tfidf_0,0,0.0,Numeriek,0,-
tfidf_1,0,0.0,Numeriek,0,-
...,...,...,...,...,...
tfidf_95,0,0.0,Numeriek,0,-
tfidf_96,0,0.0,Numeriek,0,-
tfidf_97,0,0.0,Numeriek,0,-
tfidf_98,0,0.0,Numeriek,0,-


In [123]:
display(df.head())

,Titel,Beschrijving,Genre,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
0,Ghosts of Mars,set second half 22nd centuri film depict mar p...,Thriller,0.146294,-0.005191,-0.104845,0.041571,-0.032765,0.007824,0.001906,...,-0.030355,0.012411,-0.021291,0.035317,-0.032484,-0.030526,-0.018135,-0.019691,0.011690,-0.033660
1,White Of The Eye,seri murder rich young women throughout arizon...,Thriller,0.153784,0.008972,-0.031852,-0.047422,-0.013832,-0.055753,-0.008687,...,0.005837,0.005480,-0.002502,0.005495,-0.021863,0.015756,0.001909,0.012628,-0.012253,0.013138
2,A Woman in Flames,eva upper class housewif becom frustrat leav a...,Drama,0.243657,0.009673,0.030758,-0.047815,0.044520,-0.042292,0.018025,...,0.034929,0.019593,-0.036985,0.007984,-0.015622,0.005637,-0.002477,-0.011050,0.004157,0.013961
3,The Sorcerer's Apprentice,everi hundr year evil morgana return claim fin...,Family Film,0.120472,-0.012611,-0.004944,0.005937,0.004540,0.041773,-0.029839,...,0.041978,-0.009046,0.003448,0.013986,-0.034088,-0.015150,0.029275,0.002203,-0.013272,0.012957
4,Little city,adam san francisco base artist work cab driver...,Romantic comedy,0.161932,-0.008120,0.047013,-0.054718,0.023333,-0.037162,-0.033990,...,0.064744,-0.013425,-0.004950,-0.070668,-0.035300,0.027528,0.123788,0.096733,-0.070733,-0.085306


In [ ]:
data_info(df)

In [ ]:
display(df.head())

Het dataframe dat wij uit onze pipeline krijgen heeft .... rijen en 3 kolommen. De kolommen bestaan uit de titel van de film, het genre, een (korte) omschrijving van de film en features .... . Sommige omschrijvingen bestaan uit een paar zinnen en sommige bestaan uit grote alinea's. Nu het dataframe af is, kan het datascience team aan de slag.

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Aantonen dat de Pipeline werkt</div>
    <a name='toon'></a>
</div>

Het voorspellen van het genre is een classificatie probleem, want je gaat de films classificeren op basis van het genre. Daarom kiezen wij voor een decisiontree model om aantetonen dat de pipeline werkt.

In [ ]:
# X en y definiëren
X = df.drop(['genre','titel', 'beschrijving'], axis=1)
y = df['genre']

# Toepassen van train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    test_size=0.25)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)

#het model fitten
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

# accuracy berekenen
accuracy = accuracy_score(y_test, y_pred)
print(f"Nauwkeurigheid: {accuracy}")

In [ ]:
param_dt= {
    'criterion': ['gini', 'entropy'],  
    'splitter': ['best', 'random'],    
    'max_depth': [None, 5, 10, 20, 30],  
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4],    
    'max_features': ['auto', 'sqrt', 'log2', None]  
}

#gridsearch toepassen zodat de beste parameters worden gekozen
gs = GridSearchCV(estimator=dt,
                    param_grid=param_dt,
                    cv=cv,
                    scoring='accuracy',
                    n_jobs=-1)
    
# Fitten van de grid search
gs.fit(X_train, y_train)

# Tonen van de beste score en parameters
print(f"Beste accuracy: {gs.best_score_}")
print(f"Beste parameters:\n{gs.best_params_}")

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Bronnen</div>
    <a name='bron'></a>
</div>

Please cite this paper if you write any papers involving the use of the data (van de Movie Summary Corpus):

    Learning Latent Personas of Film Characters
    David Bamman, Brendan O'Connor, and Noah A. Smith
    ACL 2013, Sofia, Bulgaria, August 2013

